In [3]:
import os
import openpyxl
import requests
from bs4 import BeautifulSoup

# Function to download a PDF from a given URL
def download_pdf(pdf_url, save_path):
    response = requests.get(pdf_url)
    if response.status_code == 200:
        with open(save_path, 'wb') as pdf_file:
            pdf_file.write(response.content)
            print(f"Downloaded: {save_path}")
    else:
        # Print both the status code and its description
        print(f"Failed to download: {save_path}")
        print(f"Status Code: {response.status_code} ({response.reason})")

# Path to your Excel file
excel_file_path = fr'C:\Users\Simon\Desktop\CBG\ReadingTaskAll.xlsx'
# Output folder for PDFs
output_folder = fr'C:\Users\Simon\Desktop\CBG\downloaded_papers'

# Load the Excel file
workbook = openpyxl.load_workbook(excel_file_path)
sheet = workbook.active

# Loop through rows and extract BibTeX citations
for row in sheet.iter_rows(min_row=2, values_only=True):
    bibtex_citation = row[10]  # Access the 11th column (index 10)

    # Search for the article on Google Scholar
    search_url = f"https://scholar.google.com/scholar?q={bibtex_citation}"
    response = requests.get(search_url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a', href=True)

        for link in links:
            href = link.get('href')
            if href.endswith('.pdf'):
                pdf_url = href
                # Create a filename based on the article's title
                title = link.text
                pdf_filename = f"{title}.pdf"
                pdf_save_path = os.path.join(output_folder, pdf_filename)

                # Download the PDF
                download_pdf(pdf_url, pdf_save_path)
                break  # Download the first PDF link found

    else:
        print(f"Failed to search for: {bibtex_citation}")
        print(f"Status Code: {response.status_code} ({response.reason})")

# Close the Excel file
workbook.close()


Downloaded: C:\Users\Simon\Desktop\CBG\downloaded_papers\[PDF] usenix.org.pdf


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Simon\\Desktop\\CBG\\downloaded_papers\\[PDF] thecvf.com.pdf'